In [1]:
import torch
from knn_loss import KNNLoss
import utility_functions

In [ ]:
def regen_rand():
    # TODO
    pass

In [2]:
tensor_a = torch.tensor([[2.0], [3.0], [4.0]])
tensor_b = torch.tensor([[0.0], [0.0], [0.0]])
tensor_c = torch.tensor([[-2.0], [-3.0], [-4.0]])

In [3]:
tensors = [tensor_a, tensor_b, tensor_c]
stack = torch.stack(tensors)
torch.mean(stack, dim = 0)
# stack

tensor([[0.],
        [0.],
        [0.]])

In [3]:
forward_pass = torch.randn(256, 128, 1)
labels = torch.rand(256, 1, 1)
labels *= 9
labels = torch.absolute(labels)
labels = torch.round(labels)

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
classes_tensor = torch.tensor(classes)
classes_tensor.shape

torch.Size([10])

In [4]:
wrap = KNNLoss(classes_tensor)

In [6]:
# wrap = KNNLoss(classes_tensor)
# div_loss = wrap.divergence_loss(forward_pass, labels)
# print(div_loss)
# centroids = wrap.get_centroids()
# print(centroids.shape)
# conv_loss = wrap.convergence_loss(forward_pass, labels)
# print(conv_loss)


the `# Calculate Current Centroids` section seem to work.

In [7]:
# print(forward_pass[0])
# print(forward_pass[1])

# a = forward_pass[0]
# b = forward_pass[1]

# cos = torch.nn.CosineSimilarity(dim = 0)
# wrap.cosine_similarity(a, b)

In [5]:
print(wrap.get_centroids())

None


In [11]:
forward_pass = utility_functions.to_device(torch.rand(256, 128, requires_grad=True), 'mps')
labels = utility_functions.to_device(torch.rand(256, 1, requires_grad=True), 'mps').mul(9).absolute().round()

In [12]:
print(labels.max())

tensor(9., device='mps:0', grad_fn=<MaxBackward1>)


In [13]:
debug_1 = utility_functions.to_device(torch.randn(len(classes_tensor), forward_pass.shape[1]), wrap.device)
print(debug_1)
wrap.centroids = debug_1

tensor([[ 0.9077, -0.8776, -0.6546,  ...,  0.2622, -1.0312,  0.3742],
        [-0.1276,  1.3103, -0.1642,  ...,  0.0772, -0.6628,  0.4404],
        [ 0.0063, -0.4420, -0.4909,  ..., -1.1859, -3.0863, -1.5780],
        ...,
        [ 0.6508, -0.5801, -0.1529,  ...,  1.1007, -1.4945, -1.4390],
        [ 0.3862, -0.3072,  0.0357,  ...,  0.3272, -1.3294,  1.3311],
        [ 0.8818,  0.9127,  1.1314,  ..., -0.1038,  0.2508,  1.2267]],
       device='mps:0')


In [14]:
c = []
for _c in wrap.classes:
    class_l = [wrap.centroids[_c].detach()]
    for inp, tar in zip(forward_pass, labels):
        if int(tar) == _c:
            class_l.append(inp)
    stacked_t = torch.stack(class_l)
    c.append(torch.mean(stacked_t, dim = 0))
wrap.centroids = torch.stack(c)
print(wrap.centroids)

tensor([[0.5220, 0.4923, 0.2852,  ..., 0.4269, 0.3404, 0.4733],
        [0.4729, 0.4295, 0.5457,  ..., 0.4143, 0.5035, 0.4679],
        [0.5379, 0.5190, 0.4447,  ..., 0.4237, 0.3033, 0.3978],
        ...,
        [0.4881, 0.4932, 0.4796,  ..., 0.5118, 0.4787, 0.4030],
        [0.3897, 0.4883, 0.5108,  ..., 0.4439, 0.4207, 0.4057],
        [0.4803, 0.4497, 0.3725,  ..., 0.4017, 0.5161, 0.6252]],
       device='mps:0', grad_fn=<StackBackward0>)


In [15]:
d = []
for centroid_a in wrap.centroids:
    local_d = []    # for each centroid collect distances to other centroids in a list. This list should be ALWAYS be 9 elements long
    for centroid_b in wrap.centroids:
        if not torch.equal(centroid_a, centroid_b):
            local_d.append(wrap.cosine_similarity(centroid_a, centroid_b))
    local_t = torch.stack(local_d)
    d.append(torch.mean(local_t))
wrap.mean_centroid_distance = torch.stack(d).abs().mean()
print(wrap.mean_centroid_distance)

tensor(0.9784, device='mps:0', grad_fn=<MeanBackward0>)


In [14]:
instance_d = []
for inp, tar in zip(forward_pass, labels):
    label = int(tar)
    corresponding_centroid = wrap.centroids[label].detach()
    instance_d.append(wrap.euclidean_distance(inp, corresponding_centroid))
d = torch.mul(torch.stack(instance_d), 1)

In [15]:
convergence = torch.mean(d)
uniformity = torch.std(d)
divergence = torch.div(1, torch.sub(1, wrap.mean_centroid_distance))

In [16]:
print(
    convergence,
    uniformity,
    divergence
)

tensor(3.2234, device='mps:0', grad_fn=<MeanBackward0>) tensor(0.1345, device='mps:0', grad_fn=<StdBackward0>) tensor(43.5569, device='mps:0', grad_fn=<DivBackward0>)


> _Now the Ratio doesn't make sense anymore..._

In [17]:
print(torch.mul(divergence, convergence.add(uniformity)))

tensor(146.2589, device='mps:0', grad_fn=<MulBackward0>)


In [29]:
inputs = [utility_functions.to_device(torch.rand(256, 128, requires_grad=True), 'mps') for i in range(195)]
targets = [utility_functions.to_device(torch.rand(256, 1, requires_grad=True), 'mps').mul(9).absolute().round() for i in range(195)]

loss = KNNLoss(classes_tensor)

In [31]:
inputs = [utility_functions.to_device(torch.rand(256, 128, requires_grad=True), 'mps') for i in range(64)]
targets = [utility_functions.to_device(torch.rand(256, 1, requires_grad=True), 'mps').mul(9).absolute().round() for i in range(64)]

loss = KNNLoss(classes_tensor)

In [32]:
for input, target in zip(inputs, targets):
    print(
        loss.loss_renewed(input, target)
    )

tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)
tensor(nan, device='mps:0', grad_fn=<MulBackward0>)


KeyboardInterrupt: 